# Imports

In [1]:
import requests
import json
import pandas as pd

from sklearn.linear_model import LinearRegression

from data_to_jupyter import time_range_query, query_to_pandas, \
                            fetch_weather_crime,call_fission_function

RUN_FROM = 'uni_wifi' #'bastion'

if RUN_FROM == 'bastion' : URL, HEADERS = 'http://fission:31001/', None
if RUN_FROM == 'uni_wifi': URL, HEADERS =  'http://172.26.135.52:9090/', {'HOST': 'fission'}

# Functions

In [13]:
def get_and_stream(api: str, station_id: str, size: int, radius_km: int, verb=False) -> pd.DataFrame:
    resp_dict = json.loads(requests.get(URL+api+f'/{station_id}/{size}/{radius_km}', headers=HEADERS).text)

    count=0
    status, token, new_data = resp_dict['Status'], resp_dict['Token'], resp_dict['Data']
    data = [new_data[i]['_source'] for i in range(len(new_data))]
    if verb : print(f'Called {api} api, fetched {len(new_data)} lines')


    while (status == 200) and (new_data != []) :
        count+=1
        resp_dict = json.loads(requests.get(URL+f'stream/'+token, headers=HEADERS).text)
        status, token, new_data = resp_dict['Status'], resp_dict['Token'], resp_dict['Data']
        if verb : print(f'Called stream {count} times, fetched {len(new_data)} new lines')
        data += [new_data[i]['_source'] for i in range(len(new_data))]

    if verb: print(f'Fetched a total of {len(data)}lines')
    return pd.DataFrame.from_records(data)

In [14]:
get_and_stream(api='crime', station_id='95003', size=8000, radius_km=800, verb=True)

Called crime api, fetched 8000 lines
Called stream 1 times, fetched 8000 new lines
Called stream 2 times, fetched 2622 new lines
Called stream 3 times, fetched 0 new lines
Fetched a total of 18622lines


,reported_date,suburb,postcode,description_1,description_2,description_3,offence_count
0,2013-02-19T00:00:00,MOUNT GAMBIER,5290,OFFENCES AGAINST THE PERSON,OTHER OFFENCES AGAINST THE PERSON,Dangerous or negligent acts,1.0
1,2013-02-21T00:00:00,GLENCOE,5291,OFFENCES AGAINST PROPERTY,PROPERTY DAMAGE AND ENVIRONMENTAL,Other property damage and environmental,1.0
2,2013-02-21T00:00:00,GLENCOE,5291,OFFENCES AGAINST PROPERTY,THEFT AND RELATED OFFENCES,Theft/Illegal Use of MV,1.0
3,2013-02-21T00:00:00,TARPEENA,5277,OFFENCES AGAINST PROPERTY,SERIOUS CRIMINAL TRESPASS,SCT - Residence,1.0
4,2013-02-24T00:00:00,MOUNT GAMBIER,5290,OFFENCES AGAINST PROPERTY,PROPERTY DAMAGE AND ENVIRONMENTAL,Other property damage and environmental,2.0
...,...,...,...,...,...,...,...
18617,2023-06-21T00:00:00,MOUNT GAMBIER,5290,OFFENCES AGAINST PROPERTY,THEFT AND RELATED OFFENCES,Receive or handle proceeds of crime,1.0
18618,2023-06-25T00:00:00,MOUNT GAMBIER,5290,OFFENCES AGAINST PROPERTY,PROPERTY DAMAGE AND ENVIRONMENTAL,Other property damage and environmental,1.0
18619,2023-06-27T00:00:00,DISMAL SWAMP,5291,OFFENCES AGAINST PROPERTY,PROPERTY DAMAGE AND ENVIRONMENTAL,Other property damage and environmental,1.0
18620,2023-06-27T00:00:00,MOUNT GAMBIER,5290,OFFENCES AGAINST THE PERSON,ROBBERY AND RELATED OFFENCES,Blackmail and extortion,1.0
